# Sampling Raster Data using Points 

In [2]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import os
import numpy as np
import rasterio as rasterio
from rasterio.plot import show
import richdem as rd
import cv2 as cv

In [3]:
#normalization function
def normalize(x):
    num= x- x.min()
    den= x.max()-x.min()
    return num/den

In [4]:
np.random.seed(123)#seed
coord_landslide_1= []
coord_landslide_0= []
blank_img= np.zeros(shape=(128, 128)) #for retrieving the mask
da_landslide= pd.DataFrame(columns= {"p/a", "x", "y","img"})
base_folder = r'C:\Users\serra\OneDrive\Documents\DATA\train'
img_folder = base_folder + "\\img"
mask_folder = base_folder + "\\mask"
for imagen_file in glob.glob(img_folder + "\\*.h5"):
    # Load h5py file
    imagen_dt = h5py.File(imagen_file, 'r')
    images = imagen_dt['img']
    img_file_name = os.path.basename(imagen_file)
    img_name = img_file_name.split(".")[0]
    mask_file = mask_folder + "\\" + img_file_name.replace("image","mask")
    mask_dt = h5py.File(mask_file, 'r')
    masks = mask_dt['mask']
    
    dem= images[:,:,13] #dem
    dem= normalize(dem) #normalizar valores
    slope= images[:,:,12]
    slope= normalize(slope) #normalizar valores
    
    k = 5 #smoothing factor
    dem_suavizado = cv.blur(dem,(k,k)) #reducción de ruido dem
    dem_richdem = rd.rdarray(dem_suavizado, no_data=-9999) #convert to rdarray
    slope_suavizado = cv.blur(slope,(k,k)) #reducción de ruido slope
    dem_curvature = rd.TerrainAttribute(dem_richdem, attrib="curvature")
    dem_curvature= normalize(dem_curvature)
    dem_planform_curvature = rd.TerrainAttribute(dem_richdem, attrib="planform_curvature")
    dem_planform_curvature= normalize(dem_planform_curvature)
    dem_profile_curvature = rd.TerrainAttribute(dem_richdem, attrib="profile_curvature")
    dem_profile_curvature= normalize(dem_profile_curvature)
    NDVI= (images[:,:,7]-images[:,:,3])/(images[:,:,7]+images[:,:,3])
    NDVI= normalize(NDVI)
    BSI= ((images[:,:,10] + images[:,:,3]) - (images[:,:,7] + images[:,:,1]) / (images[:,:,10] + images[:,:,3]) + (images[:,:,7] + images[:,:,1]))
    BSI= normalize(BSI)
    bands= np.array([dem_suavizado, slope_suavizado, dem_curvature, dem_planform_curvature, dem_profile_curvature, NDVI, BSI])
    
    
    if np.count_nonzero(masks) >= 50:
        l = 50
    else:
        l = np.count_nonzero(masks)
    if l==0: continue
    print(imagen_file)
    
    new_img_np_0= np.ma.array(blank_img, mask=masks)
    xt,yt = np.where(new_img_np_0.mask <=False)  # select only non landslide values
    for j in range(l):
        k= np.random.randint(len(xt))  # select random locations
        random_pos = [xt[k], yt[k]]  # random locations
        coord_landslide_0.append(random_pos)  # list of coordinates
        indice = len(da_landslide)
        da_landslide.loc[indice, ["x"]] = random_pos[0]  # table
        da_landslide.loc[indice, ["y"]] = random_pos[1]  # table
        
        for i in range(len(bands)):
            img_np = bands[i, :, :]
            da_landslide.loc[indice, [i]] = img_np[
                coord_landslide_0[j][0], coord_landslide_0[j][1]]  # extract the values and put them in the table
        da_landslide.loc[indice, ["p/a"]] = 0  # landslide absence (table)
        da_landslide.loc[indice, ["img"]] = img_name  # landslide absence (table)
    
    new_img_np_1 = np.ma.array(blank_img, mask=np.logical_not(masks))
    xf,yf = np.where(new_img_np_1.mask <=False) #select only landslide values
    for j in range(l):
        k = np.random.randint(len(xf))  # select random locations
        random_pos = [xf[k], yf[k]]  # random locations
        coord_landslide_1.append(random_pos)  # list of coordinates
        indice = len(da_landslide)
        da_landslide.loc[indice, ["x"]] = random_pos[0]  # table
        da_landslide.loc[indice, ["y"]] = random_pos[1]  # table
        for i in range(len(bands)):
            img_np = bands[i, :, :]
            da_landslide.loc[indice, [i]] = img_np[
                coord_landslide_1[j][0], coord_landslide_1[j][1]]  # extract the values and put them in the table
        da_landslide.loc[indice, ["p/a"]] = 1  # landslide presence (table)
        da_landslide.loc[indice, ["img"]] = img_name

da_landslide.to_csv("data_extraction.csv",decimal=".", sep =";")
print(da_landslide)


Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
C:\Users\serra\OneDrive\Documents\DATA\train\img\image_1.h5
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warn

C:\Users\serra\AppData\Local\Temp\ipykernel_7720\2144108315.py:5: RuntimeWarning: invalid value encountered in true_divide
  return num/den


Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top 

C:\Users\serra\AppData\Local\Temp\ipykernel_7720\2144108315.py:5: RuntimeWarning: invalid value encountered in true_divide
  return num/den


Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top 

...

C:\Users\serra\OneDrive\Documents\DATA\train\img\image_3326.h5
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
W

C:\Users\serra\AppData\Local\Temp\ipykernel_7720\1522306259.py:34: RuntimeWarning: invalid value encountered in true_divide
  NDVI= (images[:,:,7]-images[:,:,3])/(images[:,:,7]+images[:,:,3])
C:\Users\serra\AppData\Local\Temp\ipykernel_7720\1522306259.py:36: RuntimeWarning: invalid value encountered in true_divide
  BSI= ((images[:,:,10] + images[:,:,3]) - (images[:,:,7] + images[:,:,1]) / (images[:,:,10] + images[:,:,3]) + (images[:,:,7] + images[:,:,1]))


Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top 

C:\Users\serra\AppData\Local\Temp\ipykernel_7720\1522306259.py:34: RuntimeWarning: invalid value encountered in true_divide
  NDVI= (images[:,:,7]-images[:,:,3])/(images[:,:,7]+images[:,:,3])
C:\Users\serra\AppData\Local\Temp\ipykernel_7720\1522306259.py:36: RuntimeWarning: invalid value encountered in true_divide
  BSI= ((images[:,:,10] + images[:,:,3]) - (images[:,:,7] + images[:,:,1]) / (images[:,:,10] + images[:,:,3]) + (images[:,:,7] + images[:,:,1]))


Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top 